In [ ]:
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
'''
Helper functions
'''

'''
Calculate the distance between two points
'''
def dist(n0,n1):
    return ((n1[0]-n0[0])**2+(n1[1]-n0[1])**2)**0.5

'''
function
   plotTruss - this function plots the optimized truss
   to test that you can read the input correctly, you can start by setting
   elem['a'] = np.ones(num['m']) to test
input:
   glb
   nodes - list of nodes as read from file
   n0 - list of indices of the node at the start of the member
   n1 - list of indices of the node at the end of the member
   a - optimized cross sectional area as an array [mx1]
'''
def plotTruss(glb, nodes, n0, n1, a, f):
    
    from mpl_toolkits import mplot3d
    import matplotlib.pyplot as plt
    
    fig = plt.figure(figsize=(12, 12))
    ax = plt.axes()
            
    for i, [n0i,n1i,fi] in enumerate(zip(n0,n1,f)):
        if fi>0:
            c = 'r'
        else:
            c = 'b'
        x = [[nodes[n0i][i], nodes[n1i][i]] for i in [0,1]]
        ax.plot(x[0],x[1], c=c,linewidth=2*np.sqrt(a[i]/np.pi)*500)

    plt.show()
    

In [ ]:
'''
function: 
   Read data from file
input:
   cwd - path to the input files
output:
   inputData - dictionary containing
       inputData['constraints']
       inputData['forces']
       inputData['members']
       inputData['nodes']
   num - dictionary containing
       num['c'] - number of constraints
       num['f'] - number of forces
       num['m'] - number of members
       num['n'] - number of nodes
       num['nDOF'] - number of degreees of freedom per node (2)
       num['DOF'] - total number of degrees of freedom (2n)
'''
def importData(cwd):
    inputDataNames =['members', 'constraints', 'nodes', 'forces']
    
    inputData = dict()
    num = dict()
    
    for dataName in inputDataNames:
        inputData[dataName] = np.genfromtxt(cwd+'/'+dataName+'.csv', delimiter=',')
        if len(np.shape(inputData[dataName])) == 1: # in case there is only one row of boundary conditions
            numDim = 1
        else:
            numDim = np.shape(inputData[dataName])[0]
        num[dataName[0]] = numDim

    num['nDOF'] = 2 # nodal DOF
    num['DOF']  = num['n']*num['nDOF'] # global DOF


    return [inputData, num]
    

In [ ]:
'''
function:
   Initialization of material and geometrical properties
input:
   inputData
   num
output:
   glb - dictionary containing DOF related quantities
       glb['F'] - initial F array [2n x 1] - 0 excpet for the force conditions in the forces.csv file
       glb['u'] - initial u array [2n x 1] - infinite except the boundary conditions in the constraints.csv file
       glb['fDOF'] - row id# of all the free degree of freedom
   elem - dictionary containing member related quantities
       elem['L'] - number of constraints
       elem['n0'] - number of forces
       elem['n1'] - number of members
'''
def initializeStruct(inputData,num):

    elem = dict()
    
    # Assemble global connectivity matrix
    elem['n0']  = np.array(inputData['members'][:,0], dtype=np.int8)

    elem['n1']  = np.array(inputData['members'][:,1], dtype=np.int8)
    
    # calculate length per element
    elem['L']   = np.array([dist(inputData['nodes'][n0], inputData['nodes'][n1]) for n0,n1 in zip(elem['n0'],elem['n1'])])
    
    glb = dict()

    # initial force vector
    glb['F'] = assembleBC(inputData['forces'],num,'F')

    # initial displacement vector
    glb['u'] = assembleBC(inputData['constraints'],num,'u')
    
    # Find the indices of the u inits that are inf (free), or not
    glb['fDOF'] = np.where(glb['u'] == -np.inf)[0] # these dofs are free
    
    return [glb, elem]
    

In [ ]:

'''
construct the adjacency matrix
input:
       num
       nodes - list of nodes as read from file
       n0 - list of indices of the node at the start of the member
       n1 - list of indices of the node at the end of the member
output:
       A - adjacency matrix [2*n by m]
'''
def assembleA(num, nodes, n0, n1):
    A = np.zeros([num['DOF'],num['m']]) # A is the adjacency matrix that is n * nDOF by m large
    for i in range(num['n']):
        for j in range(num['m']):
            
            if n0[j] == i: # starting node of the bar
                nS = nodes[n0[j]] # get nodal coordinate of the starting node
                nE = nodes[n1[j]] # get nodal coordinate of the ending node
                
                if nE[1] - nS[1] == 0: # if the y coordinates are the same, then bar is aligned with the x-axis
                    x = 1
                    y = 0
                elif nE[0] - nS[0] == 0: # if the x coordinates are the same, then bar is aligned with the y-axis
                    x = 0
                    y = 1
                else:
                    a = np.arctan2(nE[1] - nS[1],nE[0] - nS[0]) # find the angle of inclination of the bar
                    x = np.cos(a) # project it to the x-axis
                    y = np.sin(a) # project it to the y-axis
                    
                A[n0[j]*2,  j] = x   # slot the coefficients into the adjacency matrix for starting node x
                A[n0[j]*2+1,j] = y   # slot the coefficients into the adjacency matrix for starding node y
                A[n1[j]*2,  j] = -x  # slot the coefficients into the adjacency matrix for ending node x
                A[n1[j]*2+1,j] = -y  # slot the coefficients into the adjacency matrix for ending node y
                
    return A

In [ ]:
'''
construct the boundary condition arrays (F or u)
input:
       inputData - array of input data (forces or constraints)
       num - number of BCs
       BC - type of BC, whether 'u' for constraints or 'F' for force
output:
       out - array [mx1]
'''
def assembleBC(inputData,num,BC):
    if BC == 'u':
        out   = np.zeros(num['DOF'])-np.inf
        numBC = num['c']
    elif BC == 'F':
        out   = np.zeros(num['DOF'])
        numBC = num['f']
    
    for i in np.arange(0,numBC):
        if numBC == 1:
            n  = inputData[0]
            d  = inputData[1:3]
            m  = inputData[3]
        else:
            n  = inputData[i,0]
            d  = inputData[i,1:3]
            m  = inputData[i,3]
        for row in np.arange(0,num['nDOF']):
                if d[row] == 1:
                    idx = int(num['nDOF']*n+row)
                    out[idx] = m
    return out

In [ ]:
'''
function
   optTruss - this function should define c, A_eq, b_eq, A_ub, b_ub, bounds, and use 
   https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html
input:
   glb
   elem
   num
   yieldStress
output:
   a_opt - optimized cross sectional area as an array [mx1]
   f_opt - element-wise forces
'''

def optTruss(glb, elem, num, yieldStress):
    minA = 0.
    maxA = 0.1
    
    f_id  = np.ix_(glb['fDOF'])
    c    = np.hstack((np.zeros(num['m']), elem['L']))

    f_id  = np.ix_(glb['fDOF'])
    A_eq = np.hstack((glb['A'][f_id], np.zeros((len(glb['fDOF']),num['m']))))
    b_eq = -glb['F'][f_id]

    A_ub = np.zeros((num['m']*2, num['m']*2))
    b_ub = np.zeros(num['m']*2)

    lb   = np.zeros(num['m']*2)
    ub   = np.zeros(num['m']*2)

    for i in np.arange(num['m']):
        A_ub[i,          i]          = -1.
        A_ub[i,          i+num['m']] = -yieldStress
        A_ub[i+num['m'], i]          =  1.
        A_ub[i+num['m'], i+num['m']] = -yieldStress

        lb[i]          = None
        ub[i]          = None
        lb[i+num['m']] = minA
        ub[i+num['m']] = maxA

    bounds= [(ubi,lbi) for ubi,lbi in zip(lb,ub)]

    from scipy.optimize import linprog
    res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds = bounds)
    f_opt = res['x'][0:num['m']]
    a_opt = res['x'][num['m']:]
    sol = res['fun']

    return [a_opt,f_opt]

In [ ]:
## Optimization of 2d Truss structures
import numpy as np

# Folder containing all your models
inputFolder='../input/'

# Sub folder containing the particular model for analysis
inputModel='10_bar'

#current working directory
cwd=inputFolder+inputModel
cwd =inputFolder+inputModel
[inputData, num] = importData(cwd)

[glb, elem] = initializeStruct(inputData, num)

glb['A'] = assembleA(num, inputData['nodes'], elem['n0'], elem['n1'])


In [ ]:
yieldStress = 70e6

[a_opt,f_opt] = optTruss(glb, elem, num, yieldStress)

In [ ]:
plotTruss(glb, inputData['nodes'], elem['n0'], elem['n1'], a_opt, f_opt)